In [ ]:
# connect to drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
cd gdrive/MyDrive/SemTech4STLD_related_code_and_data/

/content/gdrive/MyDrive/SemTech4STLD_related_code_and_data


In [ ]:
# upload dataset to drive
from google.colab import files
files.upload()

In [ ]:
ls

'Copy of ClaimCiteRetrieval_ESR_based_JSONed_20-02-2024_new.xlsx'
'Copy of ClaimCiteRetrieval_ESR_based_JSONed_20-02-2024.xlsx'
 sample-dataset-for-results-analysis-SemTech4stld.xlsx


In [ ]:
import pandas as pd
# Read xl file file
csv_file_path = './Copy of ClaimCiteRetrieval_ESR_based_JSONed_20-02-2024_new.xlsx'
df = pd.read_excel(csv_file_path)

In [ ]:
df.head(2)

,APnum,PAnum,APclaim,PAclaim,APabstract,PAabstract,APipc,PAipc,PAparagraphs,PApragraphs_Json,...,PApredGPT_top2,PApredGPT_top3,PApredBARD_all3,PApredBARD_top1,PApredBARD_top2,PApredBARD_top3,APipc_GPTpred,APipc_BARDpred,APipc_abstract_GPTpred,APipc_abstract_BARDpred
0,EP4300511,US20140019162A1,Text classification method to aid on clinical ...,"An automated triage system, comprising:\na pat...",The present application describes an automatic...,An automated triage system may include a patie...,"G16H50/20, G16H10/60, G16H40/20, G06F40/10, G0...","G06F 9/00, G06O 50/24,",[0057]\nFIGS. 3A-3B illustrate a more detailed...,"{\n ""sentences"": [\n {\n ""id"": ""0057""...",...,NaN,NaN,"\n(0058, 0059, 0092)",NaN,NaN,NaN,"G16H 30/20, G06Q 50/24, A61B 5/00, G06F 19/00,...","ipc1: G06F19/00, ipc2: G06N3/08, ipc3: G16H50/...","G06F17/305, G16H30/20, A61B5/1503, G06N3/00, G...","G06F17/30, G06F7/07, G16H50/20, G16H50/50, G06..."
1,EP4273738,US20080091405A1,A semantic representation generation method of...,A method of analyzing a sentence in a source l...,In a semantic representation generation method...,A method and computer system for analyzing sen...,"G06F40/30, // G06F40/211, G06F40/268, ...",G06F 7/28,\n[0004]\nThe acquired ability to understand...,"{\n ""sentences"": [\n {\n ""id"": ""0004""...",...,NaN,NaN,"(0004, 0005, 0015)",NaN,NaN,NaN,"G06F 17/27, G06F 17/28, G06F 17/30, G06F 17/50...","\nipc1: G06N3/05, ipc2: G10L15/00, ipc3: H04L2...","G06F17/27, G06N3/04, G06F17/305, G06Q10/08, G0...","G10L15/00, G10L15/02, G10L25/00, G10L25/04, G0..."


In [ ]:
df.shape

(49, 23)

# Task 1: Length of text (no. words) across different text type of 'ClaimCiteRetrieval' dataset

In [ ]:
# Assuming df is your DataFrame
columns_to_analyze = ['APabstract', 'PAabstract', 'APclaim', 'PAclaim', 'PAparagraphs']

# Calculate number of words for each text in the specified columns
word_lengths = df[columns_to_analyze].apply(lambda x: x.apply(lambda text: len(str(text).split())))

# Display min, max, and mean word lengths
print("Minimum Word Lengths:")
print(word_lengths.min())

print("\nMaximum Word Lengths:")
print(word_lengths.max())

print("\nAverage Word Lengths:")
print(word_lengths.mean())


Minimum Word Lengths:
APabstract       23
PAabstract       58
APclaim          51
PAclaim          58
PAparagraphs    386
dtype: int64

Maximum Word Lengths:
APabstract       201
PAabstract       216
APclaim          337
PAclaim          421
PAparagraphs    1581
dtype: int64

Average Word Lengths:
APabstract      121.265306
PAabstract      137.632653
APclaim         166.163265
PAclaim         151.795918
PAparagraphs    946.387755
dtype: float64


### Task 1.1: Percentage of counts for each distinct two-letter prefix

Below code clearly says that, patent document from US and China in general holds longer abstracts and shorter claims in comparison to European patent documents.

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with the 'PAnum' column
# Replace 'PAnum' with the actual column name in your DataFrame
distinct_prefix_counts = df['PAnum'].str[:2].value_counts(normalize=True) * 100

print("Percentage of counts for each distinct two-letter prefix:")
print(distinct_prefix_counts)

Percentage of counts for each distinct two-letter prefix:
US    46.938776
CN    40.816327
WO     8.163265
EP     2.040816
KR     2.040816
Name: PAnum, dtype: float64


In [ ]:
ls

'Copy of ClaimCiteRetrieval_ESR_based_JSONed_20-02-2024_new.xlsx'
'Copy of ClaimCiteRetrieval_ESR_based_JSONed_20-02-2024.xlsx'
 sample-dataset-for-results-analysis-SemTech4stld.xlsx


# Task 2: Preprocessing for Patent classification using ChatGPT and google Gemini

### Task 2.1: Cleaning IPC columns

In [ ]:
import pandas as pd

# Columns to clean
columns_to_clean = [' APipc', 'APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to clean IPC lists
def clean_ipc_list(ipc_list):
    return [code.replace('//', '').replace(' ', '').replace('ipc1:', '').replace('ipc2:', '').replace('ipc3:', '').replace('ipc4:', '').replace('ipc5:', '').strip() for code in ipc_list]

# Apply the cleaning function to specified columns
for column in columns_to_clean:
    df[column] = df[column].apply(lambda x: clean_ipc_list(x.split(',')) if isinstance(x, str) else x)

# Print the updated DataFrame



In [ ]:
df.head(2)

,APnum,PAnum,APclaim,PAclaim,APabstract,PAabstract,APipc,PAipc,PAparagraphs,PApragraphs_Json,...,PApredGPT_top2,PApredGPT_top3,PApredBARD_all3,PApredBARD_top1,PApredBARD_top2,PApredBARD_top3,APipc_GPTpred,APipc_BARDpred,APipc_abstract_GPTpred,APipc_abstract_BARDpred
0,EP4300511,US20140019162A1,Text classification method to aid on clinical ...,"An automated triage system, comprising:\na pat...",The present application describes an automatic...,An automated triage system may include a patie...,"[G16H50/20, G16H10/60, G16H40/20, G06F40/10, G...","G06F 9/00, G06O 50/24,",[0057]\nFIGS. 3A-3B illustrate a more detailed...,"{\n ""sentences"": [\n {\n ""id"": ""0057""...",...,NaN,NaN,"\n(0058, 0059, 0092)",NaN,NaN,NaN,"[G16H30/20, G06Q50/24, A61B5/00, G06F19/00, A6...","[G06F19/00, G06N3/08, G16H50/00, A61B5/04, G06...","[G06F17/305, G16H30/20, A61B5/1503, G06N3/00, ...","[G06F17/30, G06F7/07, G16H50/20, G16H50/50, G0..."
1,EP4273738,US20080091405A1,A semantic representation generation method of...,A method of analyzing a sentence in a source l...,In a semantic representation generation method...,A method and computer system for analyzing sen...,"[G06F40/30, G06F40/211, G06F40/268, G06F40/284...",G06F 7/28,\n[0004]\nThe acquired ability to understand...,"{\n ""sentences"": [\n {\n ""id"": ""0004""...",...,NaN,NaN,"(0004, 0005, 0015)",NaN,NaN,NaN,"[G06F17/27, G06F17/28, G06F17/30, G06F17/50, G...","[G06N3/05, G10L15/00, H04L29/06, G06F17/53, G0...","[G06F17/27, G06N3/04, G06F17/305, G06Q10/08, G...","[G10L15/00, G10L15/02, G10L25/00, G10L25/04, G..."


### Task 2.2: Verification code for section-level classification

In [ ]:
# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:4]

# Function to calculate accuracy
def calculate_accuracy(truth_list, predicted_list):
    # Extract class levels from the first item in the predicted list
    predicted_class = extract_class(predicted_list[0])#0 means Top1 # change here for Top2,3

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Example usage
truth_list = ['G06F19/00', 'H04L12/26', 'G06Q50/00']
predicted_list = ['G06J18/00', 'H04L12/30', 'G06Q49/00']

# Calculate accuracy
accuracy = calculate_accuracy(truth_list, predicted_list)

# Print the accuracy
print(f'Top1 accuracy at sub-class level: {accuracy}')

Top1 accuracy at sub-class level: False


### Task 2.3: Verification code for sub-group level classification

In [ ]:
# Function to extract class level from IPC code
#def extract_class(ipc_code):
#    return ipc_code.split('/')[0][:4]

# Function to calculate accuracy
def calculate_accuracy(truth_list, predicted_list):
    # Extract class levels from the first item in the predicted list
    predicted_class = predicted_list[2] # 2 means Top 3

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(truth_code == predicted_class for truth_code in truth_list)

# Example usage
truth_list = ['G06F19/00', 'H04L12/26', 'G06Q49/00']
predicted_list = ['G06J18/00', 'H04L12/30', 'G06Q49/00']

# Calculate accuracy
accuracy = calculate_accuracy(truth_list, predicted_list)

# Print the accuracy
print(f'Top3 accuracy at sub-group level: {accuracy}')

Top3 accuracy at sub-group level: True


# Task 3: Patent classification using ChatGPT and Google Gemini

### Task 3.1: Top1 accuracies

*At section level*


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# ... (your DataFrame initialization)

# Columns to compare
truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract section level from IPC code
def extract_section(ipc_code):
    return ipc_code.split('/')[0][0] #section level

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Extract section levels from the first item in the predicted list
    predicted_section = extract_section(predicted_list[0]) #top1

    # Check if the section level of the predicted list matches any section level in the truth list
    return any(extract_section(truth_code) == predicted_section for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top1 accuracy of {predicted_column} at section level: {accuracy:.2%}')

Top1 accuracy of APipc_GPTpred at section level: 100.00%
Top1 accuracy of APipc_BARDpred at section level: 100.00%
Top1 accuracy of APipc_abstract_GPTpred at section level: 95.92%
Top1 accuracy of APipc_abstract_BARDpred at section level: 100.00%


*At class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:3]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[0])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top1 accuracy of {predicted_column} at class level: {accuracy:.2%}')

Top1 accuracy of APipc_GPTpred at class level: 100.00%
Top1 accuracy of APipc_BARDpred at class level: 100.00%
Top1 accuracy of APipc_abstract_GPTpred at class level: 95.92%
Top1 accuracy of APipc_abstract_BARDpred at class level: 79.59%


*At sub-class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:4]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[0])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top1 accuracy of {predicted_column} at sub-class level: {accuracy:.2%}')

Top1 accuracy of APipc_GPTpred at sub-class level: 48.98%
Top1 accuracy of APipc_BARDpred at sub-class level: 89.80%
Top1 accuracy of APipc_abstract_GPTpred at sub-class level: 59.18%
Top1 accuracy of APipc_abstract_BARDpred at sub-class level: 75.51%


*At sub-group level*

In [ ]:
import pandas as pd


truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']


# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Extract sub-group levels from the first item in the predicted list
    predicted_subclass = predicted_list[0]

    # Check if the sub-group level of the predicted list matches any sub-class level in the truth list
    return any(truth_code == predicted_subclass for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top1 accuracy of {predicted_column} at sub-group level: {accuracy:.2%}')


Top1 accuracy of APipc_GPTpred at sub-group level: 10.20%
Top1 accuracy of APipc_BARDpred at sub-group level: 2.04%
Top1 accuracy of APipc_abstract_GPTpred at sub-group level: 4.08%
Top1 accuracy of APipc_abstract_BARDpred at sub-group level: 0.00%


#### Task 3.2: Top2 accuracies

In [ ]:
df.head(2)

,APnum,PAnum,APclaim,PAclaim,APabstract,PAabstract,APipc,PAipc,PAparagraphs,PApragraphs_Json,...,PApredGPT_top1,PApredGPT_top2,PApredGPT_top3,PApredBARD_top1,PApredBARD_top2,PApredBARD_top3,APipc_GPTpred,APipc_BARDpred,APipc_abstract_GPTpred,APipc_abstract_BARDpred
0,EP4300511,US20140019162A1,Text classification method to aid on clinical ...,"An automated triage system, comprising:\na pat...",The present application describes an automatic...,An automated triage system may include a patie...,"[G16H50/20, G16H10/60, G16H40/20, G06F40/10, G...","G06F 9/00, G06O 50/24,",[0057]\nFIGS. 3A-3B illustrate a more detailed...,"{\n ""sentences"": [\n {\n ""id"": ""0057""...",...,NaN,NaN,NaN,NaN,NaN,NaN,"[G16H30/20, G06Q50/24, A61B5/00, G06F19/00, A6...","[G06F19/00, G06N3/08, G16H50/00, A61B5/04, G06...","[G06F17/305, G16H30/20, A61B5/1503, G06N3/00, ...","[G06F17/30, G06F7/07, G16H50/20, G16H50/50, G0..."
1,EP4273738,US20080091405A1,A semantic representation generation method of...,A method of analyzing a sentence in a source l...,In a semantic representation generation method...,A method and computer system for analyzing sen...,"[G06F40/30, G06F40/211, G06F40/268, G06F40/284...",G06F 7/28,\n[0004]\nThe acquired ability to understand...,"{\n ""sentences"": [\n {\n ""id"": ""0004""...",...,NaN,NaN,NaN,NaN,NaN,NaN,"[G06F17/27, G06F17/28, G06F17/30, G06F17/50, G...","[G06N3/05, G10L15/00, H04L29/06, G06F17/53, G0...","[G06F17/27, G06N3/04, G06F17/305, G06Q10/08, G...","[G10L15/00, G10L15/02, G10L25/00, G10L25/04, G..."


*At section level*

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# ... (your DataFrame initialization)

# Columns to compare
truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract section level from IPC code
def extract_section(ipc_code):
    return ipc_code.split('/')[0][0] #section level

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Extract section levels from the first item in the predicted list
    predicted_section = extract_section(predicted_list[1]) #top1

    # Check if the section level of the predicted list matches any section level in the truth list
    return any(extract_section(truth_code) == predicted_section for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top2 accuracy of {predicted_column} at section level: {accuracy:.2%}')

Top2 accuracy of APipc_GPTpred at section level: 100.00%
Top2 accuracy of APipc_BARDpred at section level: 89.80%
Top2 accuracy of APipc_abstract_GPTpred at section level: 100.00%
Top2 accuracy of APipc_abstract_BARDpred at section level: 100.00%


*At class-level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:3]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[1])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top2 accuracy of {predicted_column} at class level: {accuracy:.2%}')

Top2 accuracy of APipc_GPTpred at class level: 100.00%
Top2 accuracy of APipc_BARDpred at class level: 83.67%
Top2 accuracy of APipc_abstract_GPTpred at class level: 97.96%
Top2 accuracy of APipc_abstract_BARDpred at class level: 95.92%


*At sub-class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:4]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[1])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top2 accuracy of {predicted_column} at sub-class level: {accuracy:.2%}')

Top2 accuracy of APipc_GPTpred at sub-class level: 75.51%
Top2 accuracy of APipc_BARDpred at sub-class level: 53.06%
Top2 accuracy of APipc_abstract_GPTpred at sub-class level: 67.35%
Top2 accuracy of APipc_abstract_BARDpred at sub-class level: 44.90%


*At sub-group level*

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Extract sub-group levels from the first item in the predicted list
    predicted_subclass = predicted_list[1]

    # Check if the sub-group level of the predicted list matches any sub-class level in the truth list
    return any(truth_code == predicted_subclass for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top2 accuracy of {predicted_column} at sub-group level: {accuracy:.2%}')

Top2 accuracy of APipc_GPTpred at sub-group level: 8.16%
Top2 accuracy of APipc_BARDpred at sub-group level: 2.04%
Top2 accuracy of APipc_abstract_GPTpred at sub-group level: 0.00%
Top2 accuracy of APipc_abstract_BARDpred at sub-group level: 4.08%


# Top3 accuracies

*At section level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][0]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[2])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top3 accuracy of {predicted_column} at section level: {accuracy:.2%}')

Top3 accuracy of APipc_GPTpred at section level: 93.88%
Top3 accuracy of APipc_BARDpred at section level: 65.31%
Top3 accuracy of APipc_abstract_GPTpred at section level: 97.96%
Top3 accuracy of APipc_abstract_BARDpred at section level: 95.92%


*At class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:3]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[2])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top3 accuracy of {predicted_column} at class level: {accuracy:.2%}')

Top3 accuracy of APipc_GPTpred at class level: 93.88%
Top3 accuracy of APipc_BARDpred at class level: 65.31%
Top3 accuracy of APipc_abstract_GPTpred at class level: 97.96%
Top3 accuracy of APipc_abstract_BARDpred at class level: 87.76%


*At sub-class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:4]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[2])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top3 accuracy of {predicted_column} at sub-class level: {accuracy:.2%}')

Top3 accuracy of APipc_GPTpred at sub-class level: 83.67%
Top3 accuracy of APipc_BARDpred at sub-class level: 48.98%
Top3 accuracy of APipc_abstract_GPTpred at sub-class level: 91.84%
Top3 accuracy of APipc_abstract_BARDpred at sub-class level: 73.47%


*At sub-group level*

In [ ]:
import pandas as pd


truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']


# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Extract sub-group levels from the first item in the predicted list
    predicted_subclass = predicted_list[2]

    # Check if the sub-group level of the predicted list matches any sub-class level in the truth list
    return any(truth_code == predicted_subclass for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top3 accuracy of {predicted_column} at sub-group level: {accuracy:.2%}')


Top3 accuracy of APipc_GPTpred at sub-group level: 0.00%
Top3 accuracy of APipc_BARDpred at sub-group level: 0.00%
Top3 accuracy of APipc_abstract_GPTpred at sub-group level: 0.00%
Top3 accuracy of APipc_abstract_BARDpred at sub-group level: 2.04%


# Top4 accuracies

*At section level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][0]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[3])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top4 accuracy of {predicted_column} at section level: {accuracy:.2%}')

Top4 accuracy of APipc_GPTpred at section level: 95.92%
Top4 accuracy of APipc_BARDpred at section level: 83.67%
Top4 accuracy of APipc_abstract_GPTpred at section level: 100.00%
Top4 accuracy of APipc_abstract_BARDpred at section level: 69.39%


*At class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:3]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[3])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top4 accuracy of {predicted_column} at class level: {accuracy:.2%}')

Top4 accuracy of APipc_GPTpred at class level: 87.76%
Top4 accuracy of APipc_BARDpred at class level: 71.43%
Top4 accuracy of APipc_abstract_GPTpred at class level: 93.88%
Top4 accuracy of APipc_abstract_BARDpred at class level: 67.35%


*At sub-class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:4]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[3])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top4 accuracy of {predicted_column} at sub-class level: {accuracy:.2%}')

Top4 accuracy of APipc_GPTpred at sub-class level: 65.31%
Top4 accuracy of APipc_BARDpred at sub-class level: 59.18%
Top4 accuracy of APipc_abstract_GPTpred at sub-class level: 42.86%
Top4 accuracy of APipc_abstract_BARDpred at sub-class level: 51.02%


*At sub-group level*

In [ ]:
import pandas as pd


truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']


# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Extract sub-group levels from the first item in the predicted list
    predicted_subclass = predicted_list[3]

    # Check if the sub-group level of the predicted list matches any sub-class level in the truth list
    return any(truth_code == predicted_subclass for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top4 accuracy of {predicted_column} at sub-group level: {accuracy:.2%}')

Top4 accuracy of APipc_GPTpred at sub-group level: 10.20%
Top4 accuracy of APipc_BARDpred at sub-group level: 8.16%
Top4 accuracy of APipc_abstract_GPTpred at sub-group level: 0.00%
Top4 accuracy of APipc_abstract_BARDpred at sub-group level: 0.00%


# Top5 accuracies

*At section level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][0]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[0])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top5 accuracy of {predicted_column} at section level: {accuracy:.2%}')

Top5 accuracy of APipc_GPTpred at section level: 100.00%
Top5 accuracy of APipc_BARDpred at section level: 100.00%
Top5 accuracy of APipc_abstract_GPTpred at section level: 95.92%
Top5 accuracy of APipc_abstract_BARDpred at section level: 100.00%


*At class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:3]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[4])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top5 accuracy of {predicted_column} at class level: {accuracy:.2%}')

Top5 accuracy of APipc_GPTpred at class level: 89.80%
Top5 accuracy of APipc_BARDpred at class level: 73.47%
Top5 accuracy of APipc_abstract_GPTpred at class level: 87.76%
Top5 accuracy of APipc_abstract_BARDpred at class level: 65.31%


*At sub-class level*

In [ ]:
import pandas as pd

truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract class level from IPC code
def extract_class(ipc_code):
    return ipc_code.split('/')[0][:4]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]


    predicted_class = extract_class(predicted_list[4])

    # Check if the class level of the predicted list matches any class level in the truth list
    return any(extract_class(truth_code) == predicted_class for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top5 accuracy of {predicted_column} at sub-class level: {accuracy:.2%}')

Top5 accuracy of APipc_GPTpred at sub-class level: 59.18%
Top5 accuracy of APipc_BARDpred at sub-class level: 55.10%
Top5 accuracy of APipc_abstract_GPTpred at sub-class level: 44.90%
Top5 accuracy of APipc_abstract_BARDpred at sub-class level: 61.22%


*At sub-group level*

In [ ]:
import pandas as pd


truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']


# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Extract sub-group levels from the first item in the predicted list
    predicted_subclass = predicted_list[4]

    # Check if the sub-group level of the predicted list matches any sub-class level in the truth list
    return any(truth_code == predicted_subclass for truth_code in truth_list)

# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top5 accuracy of {predicted_column} at sub-group level: {accuracy:.2%}')

Top5 accuracy of APipc_GPTpred at sub-group level: 2.04%
Top5 accuracy of APipc_BARDpred at sub-group level: 0.00%
Top5 accuracy of APipc_abstract_GPTpred at sub-group level: 0.00%
Top5 accuracy of APipc_abstract_BARDpred at sub-group level: 2.04%


## Additional verification code

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# ... (your DataFrame initialization)

# Columns to compare
truth_column = ' APipc'
predicted_columns = ['APipc_GPTpred', 'APipc_BARDpred', 'APipc_abstract_GPTpred', 'APipc_abstract_BARDpred']

# Function to extract section level from IPC code
def extract_section(ipc_code):
    return ipc_code.split('/')[0][0]

# Function to calculate accuracy
def calculate_accuracy(row, predicted_column):
    truth_list = row[truth_column]
    predicted_list = row[predicted_column]

    # Check if predicted_list has enough elements
    if len(predicted_list) > 0:
        # Extract section levels from the first item in the predicted list
        predicted_section = extract_section(predicted_list[0])

        # Check if the section level of the predicted list matches any section level in the truth list
        return any(extract_section(truth_code) == predicted_section for truth_code in truth_list)
    else:
        # Print information about rows with predicted_list length less than 2
        print(f"Row with predicted_list length less than 2: {row.name}")

        # Handle the case when predicted_list does not have enough elements
        return False


# Apply the accuracy calculation function to each predicted column and calculate the mean accuracy
for predicted_column in predicted_columns:
    accuracy = df.apply(lambda row: calculate_accuracy(row, predicted_column), axis=1).mean()

    # Print the accuracy for each predicted column
    print(f'Top2 accuracy of {predicted_column} at section level: {accuracy:.2%}')


Top2 accuracy of APipc_GPTpred at section level: 100.00%
Top2 accuracy of APipc_BARDpred at section level: 100.00%
Top2 accuracy of APipc_abstract_GPTpred at section level: 95.92%
Top2 accuracy of APipc_abstract_BARDpred at section level: 100.00%


Code to avergae classification accuracies

In [ ]:
import numpy as np
import pandas as pd

# Provided values
data = np.array([
    [100, 65.31, 61.22, 2.04],
    [69.39, 67.35, 51.02, 0.00],
    [95.92, 87.76, 73.47, 2.04],
    [100, 95.92, 44.90, 4.08],
    [100, 79.59, 75.51, 0.00]
])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Column1', 'Column2', 'Column3', 'Column4'])

# Row-wise average
row_avg = df.mean(axis=1)

# Column-wise average
col_avg = df.mean(axis=0)

# Display results
print("Row-wise averages:")
print(row_avg)

print("\nColumn-wise averages:")
print(col_avg)


Row-wise averages:
0    57.1425
1    46.9400
2    64.7975
3    61.2250
4    63.7750
dtype: float64

Column-wise averages:
Column1    93.062
Column2    79.186
Column3    61.224
Column4     1.632
dtype: float64


In [ ]:
import numpy as np
import pandas as pd

# Provided values
data = np.array([
    [95.92, 95.92, 59.18, 4.08],
    [100, 97.96, 67.35, 0.00],
    [97.96, 97.96, 91.84, 0.00],
    [100, 93.88, 42.86, 0.00],
    [95.92, 87.76, 44.90, 0.00]
])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Column1', 'Column2', 'Column3', 'Column4'])

# Row-wise average
row_avg = df.mean(axis=1)

# Column-wise average
col_avg = df.mean(axis=0)

# Display results
print("Row-wise averages:")
print(row_avg)

print("\nColumn-wise averages:")
print(col_avg)


Row-wise averages:
0    63.7750
1    66.3275
2    71.9400
3    59.1850
4    57.1450
dtype: float64

Column-wise averages:
Column1    97.960
Column2    94.696
Column3    61.226
Column4     0.816
dtype: float64


In [ ]:
import numpy as np
import pandas as pd

# Provided values
data = np.array([
    [100, 100, 89.80, 2.04],
    [89.88, 83.67, 53.06, 2.04],
    [65.31, 65.31, 48.98, 0.00],
    [83.67, 71.43, 59.18, 8.16],
    [100, 73.47, 55.10, 0.00]
])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Column1', 'Column2', 'Column3', 'Column4'])

# Row-wise average
row_avg = df.mean(axis=1)

# Column-wise average
col_avg = df.mean(axis=0)

# Display results
print("Row-wise averages:")
print(row_avg)

print("\nColumn-wise averages:")
print(col_avg)


Row-wise averages:
0    72.9600
1    57.1625
2    44.9000
3    55.6100
4    57.1425
dtype: float64

Column-wise averages:
Column1    87.772
Column2    78.776
Column3    61.224
Column4     2.448
dtype: float64


In [ ]:
import numpy as np
import pandas as pd

# Provided values
data = np.array([
    [100, 100, 48.98, 10.20],
    [100, 100, 75.51, 8.16],
    [93.88, 93.88, 83.67, 0.00],
    [95.92, 87.76, 65.31, 10.20],
    [100, 89.80, 59.18, 2.04]
])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Column1', 'Column2', 'Column3', 'Column4'])

# Row-wise average
row_avg = df.mean(axis=1)

# Column-wise average
col_avg = df.mean(axis=0)

# Display results
print("Row-wise averages:")
print(row_avg)

print("\nColumn-wise averages:")
print(col_avg)


Row-wise averages:
0    64.7950
1    70.9175
2    67.8575
3    64.7975
4    62.7550
dtype: float64

Column-wise averages:
Column1    97.960
Column2    94.288
Column3    66.530
Column4     6.120
dtype: float64
